## Capstone project - week 3: Segmenting and Clustering Toronto #1

#### Import Libraries

In [115]:
!pip install beautifulsoup4
!pip install lxml

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [116]:
# import required modules
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Scraping the Wikipedia page

In [117]:
# assign link to Wiki page to variable
wiki_Toronto_postal_codes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [118]:
# get page text and parse using BeautifulSoup
source = requests.get(wiki_Toronto_postal_codes).text
soup = BeautifulSoup(source, 'lxml')

#### Locate Table and use tags to find postal code by Borough and Neighbourhood

In [119]:
# table with data
pcode_table = soup.find('table',{'class':'wikitable sortable'})
table_data = []
# find all table rows
for tr in pcode_table.find_all('tr'):
    row = []
    # find all cells within row
    for td in tr.find_all('td'):
        # append extracted and trimmed cell text into row data  
        row.append(td.get_text(strip=True))
    # skip adding row to table_data in case is empty (header row)
    if len(row):
        table_data.append(row)
# create data frame from list of lists
df_wiki = pd.DataFrame(data=table_data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
# filter out rows with Borough equal to 'Not assigned'
df_wiki = df_wiki[df_wiki.Borough != 'Not assigned']
df_wiki.head(15)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


#### Remove empty datapoint such as 'Not assigned

In [121]:
df_wiki.loc[df_wiki['Neighbourhood'] == 'Not assigned','Neighbourhood'] = df_wiki['Borough']

In [122]:
df_grouped = df_wiki.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(lambda neighbourhoods: ', '.join(neighbourhoods)).to_frame().reset_index()
df_grouped.head(30)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [123]:
# df_grouped shape
print('Data frame shape: ', df_grouped.shape)

Data frame shape:  (103, 3)
